In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple
import time
from preconditioned_cg import pcg, jacobi_preconditioner, block_jacobi_preconditioner, ssor_preconditioner, approximate_inverse_preconditioner

def create_test_matrix(n: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Creates a more challenging test problem with controlled condition number
    """
    # Create random sparse matrix with specific pattern
    A = np.zeros((n, n))
    
    # Add tridiagonal pattern
    for i in range(n):
        A[i, i] = 4.0
        if i > 0:
            A[i, i-1] = -1.0
            A[i-1, i] = -1.0
        if i < n-1:
            A[i, i+1] = -1.0
            A[i+1, i] = -1.0
            
    # Add some random perturbations to make it more challenging
    np.random.seed(42)
    R = sparse.random(n, n, density=0.01).toarray()
    A = A + 0.1 * (R + R.T)
    
    # Create solution and right-hand side
    x_true = np.random.randn(n)
    b = A @ x_true
    
    return A, b

def benchmark_preconditioners(sizes: List[int] = [500, 1000, 2000], 
                            tol: float = 1e-10,
                            max_iter: int = 1000) -> Dict:
    """
    Benchmark different preconditioners with convergence history
    """
    preconditioners = {
        'Jacobi': jacobi_preconditioner,
        'SSOR': lambda A: ssor_preconditioner(A, omega=1.5),
        'Block Jacobi': lambda A: block_jacobi_preconditioner(A, block_size=4)
    }
    
    results = {
        'sizes': sizes,
        'iterations': {name: [] for name in preconditioners},
        'times': {name: [] for name in preconditioners},
        'residuals': {name: [] for name in preconditioners},
        'convergence_history': {name: {size: [] for size in sizes} for name in preconditioners}
    }
    
    for n in sizes:
        print(f"\nTesting size {n}x{n}")
        A, b = create_test_matrix(n)
        
        # Compute and print condition number
        eigvals = np.linalg.eigvals(A)
        cond = np.abs(np.max(eigvals)/np.min(eigvals))
        print(f"Matrix condition number: {cond:.2e}")
        
        for name, precond_fn in preconditioners.items():
            print(f"\nRunning {name} preconditioner...")
            
            # Create preconditioner
            t0 = time.time()
            M = precond_fn(A)
            setup_time = time.time() - t0
            
            # Solve with PCG
            t0 = time.time()
            _, res_history, converged = pcg(A, b, M, tol=tol, max_iter=max_iter)
            solve_time = time.time() - t0
            
            # Store results
            n_iters = len(res_history) - 1
            results['iterations'][name].append(n_iters)
            results['times'][name].append(setup_time + solve_time)
            results['residuals'][name].append(res_history[-1])
            results['convergence_history'][name][n].extend(res_history)
            
            print(f"{name} results:")
            print(f"  Iterations: {n_iters}")
            print(f"  Setup time: {setup_time:.3f}s")
            print(f"  Solve time: {solve_time:.3f}s")
            print(f"  Final residual: {res_history[-1]:.2e}")
    
    return results

def plot_benchmark_results(results: Dict):
    """
    Create detailed visualization of benchmark results
    """
    sizes = results['sizes']
    
    # Create figure with subplots
    fig = plt.figure(figsize=(15, 10))
    gs = fig.add_gridspec(2, 2)
    
    # Plot iterations
    ax1 = fig.add_subplot(gs[0, 0])
    for name, iters in results['iterations'].items():
        ax1.plot(sizes, iters, 'o-', label=name)
    ax1.set_xlabel('Matrix Size')
    ax1.set_ylabel('Iterations')
    ax1.set_title('Convergence Speed')
    ax1.grid(True)
    ax1.legend()
    
    # Plot times
    ax2 = fig.add_subplot(gs[0, 1])
    for name, times in results['times'].items():
        ax2.plot(sizes, times, 'o-', label=name)
    ax2.set_xlabel('Matrix Size')
    ax2.set_ylabel('Total Time (s)')
    ax2.set_title('Computational Cost')
    ax2.grid(True)
    ax2.legend()
    
    # Plot convergence history for largest matrix
    ax3 = fig.add_subplot(gs[1, :])
    max_size = max(sizes)
    for name in results['convergence_history'].keys():
        history = results['convergence_history'][name][max_size]
        ax3.semilogy(history, label=name)
    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('Residual Norm (log scale)')
    ax3.set_title(f'Convergence History for {max_size}x{max_size} Matrix')
    ax3.grid(True)
    ax3.legend()
    
    plt.tight_layout()
    plt.show()

# Run benchmark
np.random.seed(42)
results = benchmark_preconditioners(
    sizes=[500, 1000, 2000],
    tol=1e-10,
    max_iter=1000
)
plot_benchmark_results(results)


Testing size 500x500
Running Jacobi preconditioner...
Jacobi: 3 iterations, 0.040 seconds
Running SSOR preconditioner...
SSOR: 3 iterations, 0.044 seconds
Running Block Jacobi preconditioner...
Block Jacobi: 3 iterations, 0.023 seconds
Running Approximate Inverse preconditioner...


/home/mo/Desktop/PCG-GNN/main/preconditioned_cg.py:104: SparseEfficiencyWarning: spilu converted its input to CSC format
  ilu = spilu(A_sparse, drop_tol=drop_tol)


Approximate Inverse: 49 iterations, 0.384 seconds

Testing size 1000x1000
Running Jacobi preconditioner...
Jacobi: 3 iterations, 0.104 seconds
Running SSOR preconditioner...
SSOR: 3 iterations, 0.194 seconds
Running Block Jacobi preconditioner...
Block Jacobi: 3 iterations, 0.090 seconds
Running Approximate Inverse preconditioner...
Approximate Inverse: 31 iterations, 0.848 seconds

Testing size 15000x15000
Running Jacobi preconditioner...
Jacobi: 2 iterations, 45.559 seconds
Running SSOR preconditioner...
SSOR: 2 iterations, 192.360 seconds
Running Block Jacobi preconditioner...
Block Jacobi: 2 iterations, 54.635 seconds
Running Approximate Inverse preconditioner...
